# Pure Python

In [1]:
import numpy as np

In [8]:
def gauss(A: np.ndarray):
    """
    Implements a simplified version of Gauss elimination method

    Parameters
    ----------
    A: numpy.ndarray
        The augmented matrix for a system of linear equations

    Returns
    -------
    A: numpy.ndarray
        The augmented matrix in row echelon form
    """
    m, n = A.shape

    for i in range(m):
        for j in range(i + 1, m):
            A[j:] = A[j:] - (A[j, i] / A[i, i]) * A[i, :]
    return A

def backwards_substitution(A: np.ndarray):
    """
    Back Substitution method for calculating the solutions of a matrix in row echelon form
    Parameters
    ----------
    A: numpy.ndarray
        A matrix in row echelon form
    swaps: list
        A list of tuples containing column swaps. These are necessary since each swap
        changes the position of a solution, so, if we want to recover the original order
        (x1, x2, ..., xn) we must pass explicitly this list of swaps.
    rref: bool
        If the input matrix is in reduced row echelon form simply take the last column.

    Returns
    -------
    numpy.ndarray
        An array containing the solutions to the system of linear equations
    """
    m, n = A.shape
    solutions = np.zeros(m)
    solutions[m - 1] = A[m - 1, n - 1] / A[m - 1, n - 2]
    for i in range(m - 2, -1, -1):
        solutions[i] = (
            A[i, n - 1] - np.dot(A[i, i + 1 : n - 1], solutions[i + 1 :])
        ) / A[i, i]
    return solutions


def solve_system(A_augmented: np.ndarray):
    return backwards_substitution(gauss(A_augmented))

In [13]:
A = np.array([[2.0, -1.0, 1.0], [-1.0, 1.0, 2.0], [1.0, 2.0, -1.0]])
b = np.array([[3.0], [7.0], [2.0]])
A_augmented = np.concatenate([A, b], axis=1)

In [14]:
solve_system(A_augmented)

array([1., 2., 3.])

In [15]:
%%timeit
solve_system(A_augmented)

8.81 µs ± 96.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# Cython

In [16]:
%load_ext cython

In [18]:
%%cython

import numpy as np

def gauss(A: np.ndarray):
    """
    Implements a simplified version of Gauss elimination method

    Parameters
    ----------
    A: numpy.ndarray
        The augmented matrix for a system of linear equations

    Returns
    -------
    A: numpy.ndarray
        The augmented matrix in row echelon form
    """
    m, n = A.shape

    for i in range(m):
        for j in range(i + 1, m):
            A[j:] = A[j:] - (A[j, i] / A[i, i]) * A[i, :]
    return A

def backwards_substitution(A: np.ndarray):
    """
    Back Substitution method for calculating the solutions of a matrix in row echelon form
    Parameters
    ----------
    A: numpy.ndarray
        A matrix in row echelon form
    swaps: list
        A list of tuples containing column swaps. These are necessary since each swap
        changes the position of a solution, so, if we want to recover the original order
        (x1, x2, ..., xn) we must pass explicitly this list of swaps.
    rref: bool
        If the input matrix is in reduced row echelon form simply take the last column.

    Returns
    -------
    numpy.ndarray
        An array containing the solutions to the system of linear equations
    """
    m, n = A.shape
    solutions = np.zeros(m)
    solutions[m - 1] = A[m - 1, n - 1] / A[m - 1, n - 2]
    for i in range(m - 2, -1, -1):
        solutions[i] = (
            A[i, n - 1] - np.dot(A[i, i + 1 : n - 1], solutions[i + 1 :])
        ) / A[i, i]
    return solutions


def solve_system_cython(A_augmented: np.ndarray):
    return backwards_substitution(gauss(A_augmented))

In [19]:
solve_system_cython(A_augmented)

array([1., 2., 3.])

In [20]:
%%timeit
solve_system_cython(A_augmented)

8.31 µs ± 25 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [46]:
%%cython -a

import cython
import numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef gauss(double[:, ::1] A):
    """
    Implements a simplified version of Gauss elimination method

    Parameters
    ----------
    A: numpy.ndarray
        The augmented matrix for a system of linear equations

    Returns
    -------
    A: numpy.ndarray
        The augmented matrix in row echelon form
    """
    cdef:
        int i, j, k
        int m = A.shape[0]
        int n = A.shape[1]

    for i in range(m):
        for j in range(i + 1, m):
            for k in range(n):
                A[j, k] = A[j, k] - (A[j, i] / A[i, i]) * A[i, k]
                
    return A


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef backwards_substitution(double[:, ::1] A):
    """
    Back Substitution method for calculating the solutions of a matrix in row echelon form
    Parameters
    ----------
    A: numpy.ndarray
        A matrix in row echelon form
    swaps: list
        A list of tuples containing column swaps. These are necessary since each swap
        changes the position of a solution, so, if we want to recover the original order
        (x1, x2, ..., xn) we must pass explicitly this list of swaps.
    rref: bool
        If the input matrix is in reduced row echelon form simply take the last column.

    Returns
    -------
    numpy.ndarray
        An array containing the solutions to the system of linear equations
    """
    cdef:
        int i, j
        int m = A.shape[0]
        int n = A.shape[1]
        double dot_product
        double[::1] solutions = np.zeros(m)
        
    solutions[m - 1] = A[m - 1, n - 1] / A[m - 1, n - 2]
    
    for i in range(m - 2, -1, -1):
        dot_product = 0
        for j in range(i + 1, n - 1):
            dot_product += A[i, j] * solutions[j]
        solutions[i] = (A[i, n - 1] - dot_product) / A[i, i]
    return solutions

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def solve_system_cython(double[:, ::1] A_augmented):
    return backwards_substitution(gauss(A_augmented))

In [54]:
%%timeit
solve_system_cython(A_augmented)

696 ns ± 9.79 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
